<a href="https://colab.research.google.com/github/LambdaTheda/CheatSheets/blob/master/pt5_R_mar19_2AM_REGRSS2_unit2_spr1_mod2_212_assmnt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
pd.set_option('display.max_rows', None)

df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [4]:
df.head(200)

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,2.0,4,2016-04-19 04:24:47,,West 18th Street,40.7429,-74.0028,7995,350 West 18th Street,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,1.0,2,2016-04-27 03:19:56,Stunning unit with a great location and lots o...,West 107th Street,40.8012,-73.9660,3600,210 West 107th Street,low,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,2.0,1,2016-04-13 06:01:42,"This huge sunny ,plenty of lights 1 bed/2 bath...",West 21st Street,40.7427,-73.9957,5645,155 West 21st Street,low,1,0,1,0,1,1,0,0,0,1,1,0,1,0,0,0,0,0,1,0,0,0,0,0
8,1.0,1,2016-04-20 02:36:35,<p><a website_redacted,Hamilton Terrace,40.8234,-73.9457,1725,63 Hamilton Terrace,medium,1,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,2.0,4,2016-04-02 02:58:15,This is a spacious four bedroom with every bed...,522 E 11th,40.7278,-73.9808,5800,522 E 11th,low,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
df

In [0]:
df.shape

In [0]:
# Train/test split. Use data from April & May 2016 to train. 

#ATTEMPT 1: getting error:
'''
File "<ipython-input-24-7a30e2bbb7ec>", line 4
    train.head()
        ^
SyntaxError: invalid syntax
'''
#train = (df[df['created']].dt.year == 2016) & df[df['created'].dt.month == 4 | df[df['created'].dt.month == 5]


In [0]:
#train = (df[df['created']].dt.year == 2016) & df[df['created'].dt.month == 4 | df[df['created'].dt.month == 5]


In [0]:
#ATTEMPT 2: WORKED using isin()

df['created'] = pd.to_datetime(df['created'])
train = df[(df['created'].dt.year == 2016) & (df['created'].dt.month.isin([4, 5]))]
train.shape


In [0]:
df['created'].value_counts().head(500)


In [0]:
#ATTEMPT 3: Parsing out only YEAR from df['created']-
df['created'] = pd.to_datetime(df['created']) #dot notation: df['created'] = pd.to_datetime(df.created); from https://www.youtube.com/watch?v=yCgJGsg0Xa4
df['year'] = df['created'].dt.year
df['year']

In [0]:
#temp df for making train set from df with df['created'].dt.year == 2016
df_train_yr = df[(df['year'] == 2016)]
df_train_yr.head(200)


In [0]:
#filter df_train_yr for April and May rows- WORKS
df_train_yr_and_months = df_train_yr[df_train_yr['created'].dt.month.isin([4, 5])]  #df[(df['year'] == 2016)]
df_train_yr_and_months.head(200)          

In [0]:
df_train_yr_and_months.sample(300)

In [0]:
df_train = df_train_yr_and_months
df_train.head(300)

In [0]:
df_train.shape

# FILTER TEST SET USING JUNE **2016**

In [0]:
# ATTEMPT 1: Use data from June 2016 to test- no rows!
'''
df_test = df_train[(df_train['created'].dt.month == 6)]
df_test.head()
'''

In [0]:
df['created'] = pd.to_datetime(df['created'])
df['created'].dtype

In [0]:
# ATTEMPT 2:
#df['created'] = pd.to_datetime(df['created'])

#df['month'] = df['created'].dt.month
'''
df_test = df_train_yr[df_train_yr['created'.dt.month] == 6] 
df_test
''' 

In [0]:
# ATTEMPT 2:
df['created'] = pd.to_datetime(df['created'])
#df['month'] = df['created'].dt.month
df_test = df_train_yr[df_train_yr['created'.dt.month] == 6] 
df_test

In [20]:
df['created'] = pd.to_datetime(df['created'])
#df['created'].dtype            # RETURNS dtype('<M8[ns]')
df_test = df_train_yr[df_train_yr['created'].dt.month.isin([6])]
df_test.head(300)

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,year
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016
11,1.0,1,2016-06-03 03:21:22,Check out this one bedroom apartment in a grea...,W. 173rd Street,40.8448,-73.9396,1675,644 W. 173rd Street,low,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016
14,1.0,1,2016-06-01 03:11:01,Spacious 1-Bedroom to fit King-sized bed comfo...,East 56th St..,40.7584,-73.9648,3050,315 East 56th St..,low,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016
24,2.0,4,2016-06-07 04:39:56,SPRAWLING 2 BEDROOM FOUND! ENJOY THE LUXURY OF...,W 18 St.,40.7391,-73.9936,7400,30 W 18 St.,medium,1,1,1,1,1,1,0,0,1,0,0,0,1,0,1,1,0,0,1,0,0,0,0,0,2016
34,1.0,2,2016-06-17 03:30:24,This desirable apartment is located in Washing...,W 171 Street,40.8440,-73.9404,2300,651 W 171 Street,low,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016
39,1.0,0,2016-06-29 04:08:35,Prime Location!! This Luxury Chelsea building ...,W 34 St.,40.7530,-73.9959,2396,360 W 34 St.,medium,1,1,1,1,1,1,1,0,1,1,1,1,1,0,1,0,0,0,0,0,0,0,0,0,2016
48,1.0,1,2016-06-21 04:09:32,<p><a website_redacted,Second Avenue,40.7802,-73.9504,1900,1731 Second Avenue,low,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016
59,1.0,1,2016-06-08 06:26:49,Steps to G TrainShared BackyardShared RoofHard...,Clifton Pl,40.6888,-73.9522,2000,310 Clifton Pl,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016
82,1.0,0,2016-06-24 05:04:03,(((Spacious 2 bedroom in the Upper East Side))...,E 61 St.,40.7617,-73.9626,2625,311 E 61 St.,low,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016


In [21]:
df_test.shape

(16973, 35)

In [0]:
'''
test = pd.date_range('01-Jan-2016', '31-Dec-2016', freq='M')
test
'''

In [0]:
df_test['created'].value_counts()

In [0]:
#df.loc[df['created'].dt.year == 2016]

In [0]:
# Use data from June 2016 to test
#test = df[(df['created'].dt.year == 2016) & (df['created'].dt.month == 'June')]
'''
test = df[(df['created'].dt.month == 'June')]

test.shape
'''

In [0]:
#df.iloc[lambda df: df['created']  == 2016]

In [0]:
#df.iloc[lambda df: df.created == 2016] #df.iloc[lambda x: x.index % 2 == 0] from https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.iloc.html

In [0]:
#train = df[(df['created'].dt.year == 2016) & (df['created'].dt.month.isin([4, 5]))]


In [0]:
#train.sample(500)


In [0]:
# ATTEMPT 2: Parsing out only YEAR from df['created']
'''
df['created'] = pd.to_datetime(df['created'])               

df['created']
'''